![capa_episodio_1_ciclomobilidade.jpg](imagem/capa_episodio_1_ciclomobilidade.jpg)


# **Introdução**

Este repositório contém as análises realizadas para se chegar às evidências que deram base ao primeiro episódio da reportagem seriada "Malha Cicloviária para quem?"

O principal achado da análise aponta para a concentração da infraestrutura cicloviária em bairros de Índice de Desenvolvimento Humano (IDH) médio, alto e muito alto, quando vista proporcionalmente à malha viária de cada bairro.

Confira abaixo a análise completa e [leia o primeiro episódio da reportagem no O POVO +](https://).

## Índice
1. [Instalação e importação de bibliotecas](#instalacao)


2. [Preparação das bases de dados](#preparacao)

  2.1 [Bairros de Fortaleza](#bairros)

  2.2 [Eixos viários](#eixos-viarios)

  2.3 [Malha cicloviária atual](#malha-cicloviaria)
  

3. [Análises](#analises)

  3.1 [Calcular proporção por IDH](#proporcao-idh)

  3.2 [Dados sobre pavimentação](#pavimentacao)

  3.3 [Informações sobre a malha cicloviária atual](#informacoes)

  3.4 [Dados sobre bairros para reportagem](#reportagem)

  3.5 [Gráfico da evolução temporal](#grafico)

  3.6 [Eixo Viário por faixa de IDH](#idh)


**EXPEDIENTE**

**Textos**

Gabriela Custódio e Thays Lavor

**Edição DATADOC**

Thays Lavor

**Edição O POVO Mais**

Fátima Sudário e Regina Ribeiro

**Front-End**

Michele Medeiros e Alexandre Cajazeira

**Análise e visualização de dados**

Alexandre Cajazeira e Gabriela Custódio

**Design**

Cristiane Frota, Isac Bernado e Luciana Pimenta

**Pesquisa**

Miguel Pontes e Roberto Araújo

# 1. Instalação e importação de bibliotecas <a name='instalacao'></a>

In [ ]:
!pip install googlemaps
!pip install geopandas
!pip install unidecode

In [2]:
import pandas as pd
import geopandas as gpd
import unidecode
import shapely
from shapely.geometry import Point, MultiPolygon
from pyproj import Geod

# 2. Preparação das bases de dados <a name='preparacao'></a>

## 2.1 Bairros de Fortaleza <a name='bairros'></a>

Neste ponto, importamos base de bairros, criamos um GeoDataFrame e calculamos
a extensão, em km, de uma interseção entre a malha viária ou cicloviaria e um polígono (bairro)

In [3]:
bases_originais = 'bases_originais/'

In [42]:
arquivos_gerados = 'arquivos_gerados/'

In [6]:
# IMPORTAR ARQUIVO COM OS BAIRROS DE FORTALEZA

bairros = bases_originais + 'outras_bases/bairrosFortaleza2019.csv'
df_bairros = pd.read_csv(bairros)

In [7]:
df_bairros.info()

# Limpar colunas realmente importantes; 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         121 non-null    object 
 1   GID         121 non-null    int64  
 2   id          121 non-null    int64  
 3   the_geom    121 non-null    object 
 4   bairro      121 non-null    object 
 5   regional    121 non-null    object 
 6   area_m2     121 non-null    float64
 7   area_ha     121 non-null    float64
 8   area_km2    121 non-null    float64
 9   perimetr_m  121 non-null    float64
 10  sr_2019     121 non-null    object 
 11  ta_2019     121 non-null    int64  
 12  distr_2010  121 non-null    object 
 13  nmhab_2010  121 non-null    int64  
 14  decreto     121 non-null    object 
 15  idhm_2010   121 non-null    float64
 16  equipsaude  121 non-null    int64  
 17  escola_est  121 non-null    int64  
 18  escola_mun  121 non-null    int64  
 19  constutela  121 non-null    i

In [8]:
def calculaFaixa(idh):
    if idh< 0.5:
        return "Muito Baixo"
    elif idh < 0.6:
        return "Baixo"
    elif idh < 0.7:
        return "Médio"
    elif idh < 0.8:
        return "Alto"    
    return "Muito Alto"

df_bairros['Faixa IDH'] = df_bairros.apply(lambda x: calculaFaixa(x['idhm_2010']), axis=1)

In [9]:
# Criando uma geoDataFrame (geopandas) utilizando a geometria dos bairros; 
df_bairros['the_geom'] = gpd.GeoSeries.from_wkt(df_bairros['the_geom'])

gBairros = gpd.GeoDataFrame(df_bairros, geometry='the_geom')

In [10]:
# from pyproj import Geod

# Função para calcular o tamanho de uma interseção (em kms) entre uma malha viária ou cicloviaria e um polígono (no caso, um bairro)
geod = Geod(ellps="WGS84")

def intersection_lenght(multiLineString, multiPolygon):
    return geod.geometry_length(multiPolygon.intersection(multiLineString))/1000

## 2.2 Eixos viários <a name='eixos-viarios'></a>

Ao final desta seção, temos um dataframe com os eixos viários da Cidade e acrescentamos ao geoDF de bairros o tamanho do eixo viário de cada bairro de Fortaleza. 

In [11]:
eixos_viarios = gpd.read_file('bases_originais/outras_bases/eixos_viarios.geojson')

eixos_viarios.head()

,Via,Tipo,Pavimentac,geometry
0,EST DO GEREBERABA,Estrada,Sem,"MULTILINESTRING ((-38.43133 -3.80506, -38.4333..."
1,R 10 (CJ JARDIM CASTELAO),Rua,Asfalto,"MULTILINESTRING ((-38.53160 -3.82197, -38.5317..."
2,R CEL ANTONIO SAMPAIO,Rua,Sem,"MULTILINESTRING ((-38.49392 -3.85621, -38.4938..."
3,AL DO CONTORNO,Alameda,Asfalto,"MULTILINESTRING ((-38.51950 -3.80791, -38.5199..."
4,R TAILANDIA,Rua,Paralelep.,"MULTILINESTRING ((-38.60480 -3.75572, -38.6047..."


In [12]:
list(eixos_viarios['Pavimentac'].unique())

['Sem',
 'Asfalto',
 'Paralelep.',
 'Pedra rust',
 'Rua Projetada',
 'Concreto',
 'Premoldado']

In [13]:
def ext_viaria_bairro(bairro_geom):
    return round(eixos_viarios['geometry'].apply(lambda x: intersection_lenght(x,bairro_geom)).sum(),2)

In [14]:
def ext_viaria_bairro_com_pavimentacao(bairro_geom):
    return round(eixos_viarios[eixos_viarios['Pavimentac'] != 'Sem']['geometry'].apply(lambda x: intersection_lenght(x,bairro_geom)).sum(),2)

In [15]:
gBairros['ext_eixos_viarios'] = gBairros['the_geom'].apply(lambda x: ext_viaria_bairro(x))

In [16]:
gBairros['ext_viaria_bairro_com_pavimentacao'] = gBairros['the_geom'].apply(lambda x: ext_viaria_bairro_com_pavimentacao(x))

In [17]:
# Calculando o tamanho em km de cada via dentro do eixo viário da cidade; 
eixos_viarios['len_gpd'] = eixos_viarios['geometry'].apply(lambda x: round(geod.geometry_length(x)/1000,2))

In [18]:
eixos_viarios.groupby(['Pavimentac'])['len_gpd'].sum()

Pavimentac
Asfalto          2994.93
Concreto            1.83
Paralelep.         59.27
Pedra rust        957.11
Premoldado          2.79
Rua Projetada      10.20
Sem               712.69
Name: len_gpd, dtype: float64

## 2.3 Malha cicloviária atual <a name='malha-cicloviaria'></a>

Aqui, fazemos a leitura dos arquivos da malha cicloviária, extraídos no dia 17 de agosto de 2022 do [mapa Malha Cicloviária de Fortaleza](https://www.google.com/maps/d/u/1/viewer?mid=1eqNX-fl3ENPC8_1tqzbRDYZFQmA&ll=-3.7712495577429186%2C-38.50590144121097&z=12), mantido pela Prefeitura de Fortaleza, calculamos a extensão do eixo cicloviário de cada um dos bairros e a proporção entre a malha cicloviária e a malha cicloviária de cada um deles.

In [19]:
ciclovias = gpd.read_file('bases_originais/malha_cicloviaria_fortaleza/ciclovias.geojson')
ciclofaixas = gpd.read_file('bases_originais/malha_cicloviaria_fortaleza/ciclofaixas.geojson')
ciclorotas = gpd.read_file('bases_originais/malha_cicloviaria_fortaleza/ciclorotas.geojson')
passeios_compartilhados = gpd.read_file('bases_originais/malha_cicloviaria_fortaleza/passeios_compartilhados.geojson')

In [20]:
# Criando uma coluna para Tipologia apenas para concatenar todos os datasets em um único dataframe contendo toda a malha cicloviária da cidade: 
ciclovias['Tipologia'] = 'Ciclovia'
ciclofaixas['Tipologia'] = 'Ciclofaixas'
ciclorotas['Tipologia'] = 'Ciclorotas'
passeios_compartilhados['Tipologia'] = 'Passeios compartilhados'

In [21]:
# Concatenar a malha cicloviária (todas as tipologias)
malha_cicloviaria_atual = pd.concat([ciclovias, ciclofaixas,ciclorotas,passeios_compartilhados])
# Calculando a extensão de cada ciclovia, ciclofaixa, ciclorrota e passeio compartilhado (para posterior comparação aos cálculos do mapa de fortaleza)
malha_cicloviaria_atual['len_gpd'] = malha_cicloviaria_atual['geometry'].apply(lambda x: round(geod.geometry_length(x)/1000,2))

In [22]:
malha_cicloviaria_atual.head(2)

,Name,description,timestamp,begin,end,altitudeMode,tessellate,extrude,visibility,drawOrder,...,Elementos,Extens__o,Data_de_im,Posi____o,Separa____o,geometry,Tipologia,Extens___o,Posi______o_na,len_gpd
0,Av. Gen. Osório de Paiva,descrição: descrição: <br>tessellate: 1<br>ext...,None,None,None,None,1,0,-1,None,...,Canteiro físico,4.0,antes de julho 2003,Canteiro central,Canteiro físico,"LINESTRING Z (-38.61807 -3.82106 0.00000, -38....",Ciclovia,NaN,NaN,3.98
1,Via Expressa,descrição: descrição: <br>tessellate: 1<br>ext...,None,None,None,None,1,0,-1,None,...,Canteiro físico e mureta de concreto,4.6,antes de julho 2003,Canteiro central,Canteiro físico,"LINESTRING Z (-38.47981 -3.72287 0.00000, -38....",Ciclovia,NaN,NaN,4.61


In [23]:
malha_cicloviaria_atual['len_gpd'].sum()

412.0899999999999

In [24]:
ciclovias['geometry'].apply(lambda x: round(geod.geometry_length(x)/1000,2)).sum()

129.84999999999997

In [25]:
ciclofaixas['geometry'].apply(lambda x: round(geod.geometry_length(x)/1000,2)).sum()

268.54

In [26]:
ciclorotas['geometry'].apply(lambda x: round(geod.geometry_length(x)/1000,2)).sum()

12.120000000000001

In [27]:
passeios_compartilhados['geometry'].apply(lambda x: round(geod.geometry_length(x)/1000,2)).sum()

1.58

In [28]:
# Esta função serve para calcular a extensão do sistema cicloviário por bairro

def ext_cicloviaria_atual_por_bairro(bairro_geom):
    return round(malha_cicloviaria_atual['geometry'].apply(lambda x: intersection_lenght(x,bairro_geom)).sum(),2)

In [29]:
# Nesta célula calculamos a extensão do eixo cicloviário de cada um dos bairros da cidade, dispostos no geodataframe gBairros
gBairros['ext_eixo_cicloviario_atual'] = gBairros['the_geom'].apply(lambda x: ext_cicloviaria_atual_por_bairro(x))

In [30]:
f'Foram encontrados {round(gBairros["ext_eixo_cicloviario_atual"].sum(),2)} de {round(malha_cicloviaria_atual["len_gpd"].sum(),2)} kms;'

'Foram encontrados 408.41 de 412.09 kms;'

In [31]:
# Proporção do eixo cicloviário pelo eixo viário de cada bairro: 

gBairros['prop_cic_total'] = round(gBairros['ext_eixo_cicloviario_atual']/gBairros['ext_eixos_viarios']*100)

In [32]:
gBairros.head(1)

,FID,GID,id,the_geom,bairro,regional,area_m2,area_ha,area_km2,perimetr_m,...,FONTE,ANO_REF,DESCRICAO,NOME,FLA_LEG,Faixa IDH,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_cic_total
0,vw_fortaleza_limites_administrativos_2020.fid-...,95,95,"MULTIPOLYGON (((-38.53142 -3.79758, -38.53125 ...",Boa Vista/Castelão,SER VI,2957500.947,295.75,2.958,8249.362,...,IPLANFOR,2020,<br><div><strong>Regional 2020: </strong>SR 8<...,Boa Vista/Castelão,Bairros de Fortaleza,Muito Baixo,27.25,22.0,3.0,11.0


In [33]:
# Filtrando as colunas importantes do geoDF gBairros 
gBairros_menor = gBairros[['id','bairro', 'the_geom', 'ext_eixos_viarios', 'ext_viaria_bairro_com_pavimentacao','ext_eixo_cicloviario_atual','prop_cic_total', 'idhm_2010','Faixa IDH']].sort_values(by='prop_cic_total', ascending=False)
gBairros_menor.head()

,id,bairro,the_geom,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_cic_total,idhm_2010,Faixa IDH
118,28,Parque Araxá,"MULTIPOLYGON (((-38.54377 -3.73727, -38.54382 ...",11.41,11.29,3.27,29.0,0.587355,Baixo
61,23,José Bonifácio,"MULTIPOLYGON (((-38.52393 -3.73690, -38.52390 ...",19.52,19.46,5.24,27.0,0.643759,Médio
44,29,Parquelândia,"MULTIPOLYGON (((-38.56415 -3.73568, -38.56407 ...",27.80,27.75,6.84,25.0,0.628401,Médio
108,5,Moura Brasil,"MULTIPOLYGON (((-38.54170 -3.71792, -38.54173 ...",8.98,8.90,2.06,23.0,0.284686,Muito Baixo
38,36,Varjota,"MULTIPOLYGON (((-38.48860 -3.73596, -38.48861 ...",13.41,12.95,2.99,22.0,0.717610,Alto


In [34]:
# Criando uma nova coluna com formatação única para os nomes dos bairros; 

lista_gBairros = gBairros_menor['bairro'].to_list()
lista_gBairros_processados = list()

for item in lista_gBairros:
    item = unidecode.unidecode(item)
    item = item.upper()
    lista_gBairros_processados.append(item)

gBairros_menor['NOME_BAIRRO'] = lista_gBairros_processados
gBairros_menor['NOME_BAIRRO'] = gBairros_menor['NOME_BAIRRO'].replace('PREFEITO JOSE VALTER', 'PREFEITO JOSE WALTER').replace('ENG LUCIANO CAVALCANTE', 'ENGENHEIRO LUCIANO CAVALCANTE').replace('BOA VISTA', 'BOA VISTA/CASTELAO').replace('SAO JOAO DO TAUAPE', 'TAUAPE')

In [35]:
gBairros_menor.head(1)

,id,bairro,the_geom,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_cic_total,idhm_2010,Faixa IDH,NOME_BAIRRO
118,28,Parque Araxá,"MULTIPOLYGON (((-38.54377 -3.73727, -38.54382 ...",11.41,11.29,3.27,29.0,0.587355,Baixo,PARQUE ARAXA


In [44]:
# 3º ARQUIVO GERADO: dataset_bairros_eixo_cicloviario_idh.geojson
gBairros_menor.to_csv(arquivos_gerados+ 'dataset_bairros_eixo_cicloviario_idh.csv', sep=';', encoding='utf-8')

In [45]:
gpd.GeoDataFrame(gBairros_menor, geometry="the_geom").to_file(arquivos_gerados+'dataset_bairros_eixo_cicloviario_idh.geojson', driver='GeoJSON')

# 3. Análises <a name='analises'></a>

##### 3.1 Calcular proporção por IDH <a name='proporcao-idh'></a>

In [46]:
# Quantos bairros por faixa de IDH? 

bairros_por_idh = gBairros_menor['Faixa IDH'].value_counts().reset_index()
bairros_por_idh

,index,Faixa IDH
0,Muito Baixo,93
1,Baixo,14
2,Médio,6
3,Alto,5
4,Muito Alto,3


In [47]:
gBairros_menor.rename(columns={'idhm_2010':"IDH",'Faixa IDH':"TIPO_IDH"}, inplace=True)

In [48]:
#Cálculo da proporção (eixo cicloviário atual / eixo viário (com e sem pavimentação))

gBairros_menor['proporcao_eixo_viario'] = round(gBairros_menor['ext_eixo_cicloviario_atual']/gBairros_menor['ext_eixos_viarios'],3)
gBairros_menor['proporcao_eixo_viario_com_pav'] = round(gBairros_menor['ext_eixo_cicloviario_atual']/gBairros_menor['ext_viaria_bairro_com_pavimentacao'],3)
gBairros_menor['IDH'] = round(gBairros_menor['IDH'],3)
gBairros_menor.head(2)

,id,bairro,the_geom,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_cic_total,IDH,TIPO_IDH,NOME_BAIRRO,proporcao_eixo_viario,proporcao_eixo_viario_com_pav
118,28,Parque Araxá,"MULTIPOLYGON (((-38.54377 -3.73727, -38.54382 ...",11.41,11.29,3.27,29.0,0.587,Baixo,PARQUE ARAXA,0.287,0.290
61,23,José Bonifácio,"MULTIPOLYGON (((-38.52393 -3.73690, -38.52390 ...",19.52,19.46,5.24,27.0,0.644,Médio,JOSE BONIFACIO,0.268,0.269


In [49]:
# quantidade de bairros por tipo de IDH / proporcao_bairros = proporção em relação ao total de 121 bairros; 

groupby_bairros_idh = gBairros_menor[['TIPO_IDH', 'bairro']].groupby('TIPO_IDH').count().sort_values(by='bairro', ascending=False).reset_index()
groupby_bairros_idh['proporcao_bairros'] = round(groupby_bairros_idh['bairro']/groupby_bairros_idh['bairro'].sum()*100,2)
groupby_bairros_idh

,TIPO_IDH,bairro,proporcao_bairros
0,Muito Baixo,93,76.86
1,Baixo,14,11.57
2,Médio,6,4.96
3,Alto,5,4.13
4,Muito Alto,3,2.48


In [50]:
# prop_cic_total = proporção do sistema cicloviário 
gBairros_menor.sort_values(by='ext_eixos_viarios', ascending=False).head(10).sort_values(by='prop_cic_total', ascending=False).head(3)

,id,bairro,the_geom,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_cic_total,IDH,TIPO_IDH,NOME_BAIRRO,proporcao_eixo_viario,proporcao_eixo_viario_com_pav
120,10,Centro,"MULTIPOLYGON (((-38.54025 -3.72842, -38.54031 ...",91.88,90.06,14.84,16.0,0.557,Baixo,CENTRO,0.162,0.165
99,65,Messejana,"MULTIPOLYGON (((-38.49275 -3.81116, -38.49236 ...",92.35,86.27,11.09,12.0,0.376,Muito Baixo,MESSEJANA,0.120,0.129
65,78,Granja Lisboa,"MULTIPOLYGON (((-38.63546 -3.79363, -38.63545 ...",95.04,78.10,7.41,8.0,0.170,Muito Baixo,GRANJA LISBOA,0.078,0.095


##### 3.2 Dados sobre pavimentação <a name='pavimentacao'></a>

In [51]:
# Comparar, de maneira ordenada, os bairros com maior diferença entre a proporção
# do sistema cicloviário considerando o eixo viário total e apenas o eixo viário com pavimentação

gBairros_menor['diferença_eixo_viario_com_sem_pavimentacao'] = gBairros_menor['proporcao_eixo_viario_com_pav'] - gBairros_menor['proporcao_eixo_viario'	]
gBairros_menor.sort_values(by='diferença_eixo_viario_com_sem_pavimentacao', ascending=False).head(3)

,id,bairro,the_geom,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_cic_total,IDH,TIPO_IDH,NOME_BAIRRO,proporcao_eixo_viario,proporcao_eixo_viario_com_pav,diferença_eixo_viario_com_sem_pavimentacao
84,70,Sabiaguaba,"MULTIPOLYGON (((-38.44600 -3.80521, -38.44612 ...",24.22,9.45,4.00,17.0,0.267,Muito Baixo,SABIAGUABA,0.165,0.423,0.258
113,69,Pedras,"MULTIPOLYGON (((-38.50171 -3.88813, -38.50177 ...",26.40,10.45,1.41,5.0,0.264,Muito Baixo,PEDRAS,0.053,0.135,0.082
78,74,Canindezinho,"MULTIPOLYGON (((-38.59151 -3.79398, -38.59144 ...",74.08,39.96,6.68,9.0,0.136,Muito Baixo,CANINDEZINHO,0.090,0.167,0.077


In [52]:
print('O total da malha viária de Fortaleza é de:', gBairros_menor['ext_eixos_viarios'].sum())

O total da malha viária de Fortaleza é de: 4689.9800000000005


In [53]:
# Pq da 408? Pq só considera a malha viária interna à geometria da cidade (+-3km estão fora dos limites oficiais da Cidade)

print('O total da malha cicloviária de Fortaleza é de:', round(gBairros_menor['ext_eixo_cicloviario_atual'].sum(),2))

O total da malha cicloviária de Fortaleza é de: 408.41


In [54]:
print('O percentual da malha cicloviária de Fortaleza em relação à malha viária da Cidade é de:', round(gBairros_menor['ext_eixo_cicloviario_atual'].sum()/gBairros_menor['ext_eixos_viarios'].sum()*100,2), '%')

O percentual da malha cicloviária de Fortaleza em relação à malha viária da Cidade é de: 8.71 %


In [55]:
print('O percentual da malha cicloviária de Fortaleza em relação à malha viária da Cidade >> com pavimentação << é de:', round(gBairros_menor['ext_eixo_cicloviario_atual'].sum()/gBairros_menor['proporcao_eixo_viario_com_pav'].sum()*100,2), '%')

O percentual da malha cicloviária de Fortaleza em relação à malha viária da Cidade >> com pavimentação << é de: 3106.49 %


In [56]:
eixo_viario_com_pav = gBairros_menor[['TIPO_IDH', 'ext_eixos_viarios', 'ext_viaria_bairro_com_pavimentacao', 'ext_eixo_cicloviario_atual']].groupby(by='TIPO_IDH').sum().reset_index()
eixo_viario_com_pav

,TIPO_IDH,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual
0,Alto,98.83,95.47,19.05
1,Baixo,447.58,420.32,67.15
2,Muito Alto,171.47,169.33,33.00
3,Muito Baixo,3778.29,3120.39,251.04
4,Médio,193.81,192.20,38.17


In [57]:
eixo_viario_com_pav['prop_total'] = round(eixo_viario_com_pav['ext_eixo_cicloviario_atual']/eixo_viario_com_pav['ext_eixos_viarios']*100,1)
eixo_viario_com_pav['prop_com_pav'] = round(eixo_viario_com_pav['ext_eixo_cicloviario_atual']/eixo_viario_com_pav['ext_viaria_bairro_com_pavimentacao']*100,1)

In [58]:
eixo_viario_com_pav

,TIPO_IDH,ext_eixos_viarios,ext_viaria_bairro_com_pavimentacao,ext_eixo_cicloviario_atual,prop_total,prop_com_pav
0,Alto,98.83,95.47,19.05,19.3,20.0
1,Baixo,447.58,420.32,67.15,15.0,16.0
2,Muito Alto,171.47,169.33,33.00,19.2,19.5
3,Muito Baixo,3778.29,3120.39,251.04,6.6,8.0
4,Médio,193.81,192.20,38.17,19.7,19.9


##### 3.3 Informações sobre a malha cicloviária atual <a name='informacoes'></a>

In [59]:
malha_cicloviaria_atual.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 393 entries, 0 to 3
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Name            393 non-null    object  
 1   description     393 non-null    object  
 2   timestamp       0 non-null      object  
 3   begin           0 non-null      object  
 4   end             0 non-null      object  
 5   altitudeMode    0 non-null      object  
 6   tessellate      393 non-null    int64   
 7   extrude         393 non-null    int64   
 8   visibility      393 non-null    int64   
 9   drawOrder       0 non-null      object  
 10  icon            0 non-null      object  
 11  descri____o     393 non-null    object  
 12  tessellate_1    0 non-null      object  
 13  extrude_1       0 non-null      object  
 14  visibility_1    0 non-null      object  
 15  Trecho          393 non-null    object  
 16  Posi____o_na    389 non-null    object  
 17  Sentido_de

In [60]:
sist_cicloviario = malha_cicloviaria_atual[['Name', 'Trecho', 'Extens__o', 'Data_de_im', 'geometry', 'Tipologia']]
sist_cicloviario.rename(columns={
    'Extens__o': 'Extensão',
    'Data_de_im': 'Data de implantação'}, inplace=True)
sist_cicloviario.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Name,Trecho,Extensão,Data de implantação,geometry,Tipologia
0,Av. Gen. Osório de Paiva,Entre Rua Cel. João Correia e o limite de Fort...,4.0,antes de julho 2003,"LINESTRING Z (-38.61807 -3.82106 0.00000, -38....",Ciclovia
1,Via Expressa,Entre Av. da Abolição e próximo à Av. Gov. Rau...,4.6,antes de julho 2003,"LINESTRING Z (-38.47981 -3.72287 0.00000, -38....",Ciclovia


In [61]:
sist_cicloviario['Data de implantação'] = sist_cicloviario['Data de implantação'].str.replace('antes de ', '/até')
sist_cicloviario['Ano_implantação'] = sist_cicloviario['Data de implantação'].str.split(pat='/', expand=True)[1]

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [62]:
lista_datas = ['set/20', 'mai/20', 'jun/20']
lista_novas_datas = list()

for a in sist_cicloviario['Data de implantação']:
    if a == 'set/20':
        a = a.replace(a, 'set/2020')
    elif a == 'mai/20':
        a = a.replace(a, 'mai/2020')
    elif a == 'jun/20':
        a = a.replace(a, 'jun/2020')
    lista_novas_datas.append(a)

In [63]:
sist_cicloviario['Data_implantação_2'] = lista_novas_datas

In [64]:
sist_cicloviario['Ano_implantação'] = sist_cicloviario['Data_implantação_2'].str.split(pat='/', expand=True)[1]

In [65]:
avanco_sist_cicloviario = sist_cicloviario[['Tipologia', 'Ano_implantação', 'Extensão']].dropna()

In [66]:
avanco_sist_cicloviario['Extensão'] = avanco_sist_cicloviario['Extensão'].str.replace(',', '.')

In [67]:
avanco_sist_cicloviario['Extensão'] = avanco_sist_cicloviario['Extensão'].astype(float)

In [68]:
avanco_sist_cicloviario.groupby('Tipologia').sum().reset_index()

,Tipologia,Extensão
0,Ciclofaixas,222.4
1,Ciclorotas,10.7
2,Ciclovia,120.8


#### 3.4 Dados sobre bairros para reportagem <a name='reportagem'></a>

In [69]:
lista_bairros = ['Aldeota', 'Vila Velha', 'Conjunto Ceará II', 'Meireles', 'Tauape', 'Moura Brasil', 'Parque Araxá', 'Bonsucesso', 'Jangurussu']
lista_dicionarios = []

for bairro in gBairros_menor['bairro']:
    if bairro in lista_bairros:
        bairro_selecionado = gBairros_menor[gBairros_menor['bairro'] == bairro]
        dict_bairros = {
            'Nome': bairro,
            'Faixa de IDH': bairro_selecionado['TIPO_IDH'].item(),
            'IDH': bairro_selecionado['IDH'].item(),
            'Eixo viário': bairro_selecionado['ext_eixos_viarios'].item(),
            'Eixo Cicloviário': bairro_selecionado['ext_eixo_cicloviario_atual'].item()}
        lista_dicionarios.append(dict_bairros)

In [70]:
lista_dicionarios

[{'Nome': 'Parque Araxá',
  'Faixa de IDH': 'Baixo',
  'IDH': 0.587,
  'Eixo viário': 11.41,
  'Eixo Cicloviário': 3.27},
 {'Nome': 'Moura Brasil',
  'Faixa de IDH': 'Muito Baixo',
  'IDH': 0.285,
  'Eixo viário': 8.98,
  'Eixo Cicloviário': 2.06},
 {'Nome': 'Bonsucesso',
  'Faixa de IDH': 'Muito Baixo',
  'IDH': 0.262,
  'Eixo viário': 50.67,
  'Eixo Cicloviário': 10.56},
 {'Nome': 'Aldeota',
  'Faixa de IDH': 'Muito Alto',
  'IDH': 0.867,
  'Eixo viário': 82.07,
  'Eixo Cicloviário': 16.26},
 {'Nome': 'Meireles',
  'Faixa de IDH': 'Muito Alto',
  'IDH': 0.953,
  'Eixo viário': 51.72,
  'Eixo Cicloviário': 10.59},
 {'Nome': 'Tauape',
  'Faixa de IDH': 'Muito Baixo',
  'IDH': 0.492,
  'Eixo viário': 51.27,
  'Eixo Cicloviário': 9.42},
 {'Nome': 'Vila Velha',
  'Faixa de IDH': 'Muito Baixo',
  'IDH': 0.272,
  'Eixo viário': 91.52,
  'Eixo Cicloviário': 3.25},
 {'Nome': 'Conjunto Ceará II',
  'Faixa de IDH': 'Muito Baixo',
  'IDH': 0.362,
  'Eixo viário': 50.45,
  'Eixo Cicloviário': 1.7

In [71]:
maiores_malhas_viarias = gBairros_menor[['bairro', 'ext_eixos_viarios', 'prop_cic_total']].sort_values('ext_eixos_viarios', ascending=False).head(10)
maiores_malhas_viarias.sort_values('prop_cic_total', ascending=False)

,bairro,ext_eixos_viarios,prop_cic_total
120,Centro,91.88,16.0
99,Messejana,92.35,12.0
65,Granja Lisboa,95.04,8.0
4,Passaré,104.34,7.0
75,Prefeito José Walter,94.71,7.0
67,Mondubim,122.60,6.0
83,Siqueira,99.30,6.0
17,Barra do Ceará,95.32,5.0
103,Jangurussu,127.91,3.0
82,Edson Queiroz,104.17,1.0


#### 3.5 Gráfico da evolução temporal <a name='grafico'></a>

In [72]:
sist_cicloviario.loc[sist_cicloviario['Ano_implantação'].isna(),'Ano_implantação']= "Não informado"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [73]:
# Esta função calcula a extensão do sistema cicloviário por ano, em cada bairro. O laço percorre todos bairros e todos os anos encontrados no dataframe sistemacicloviario

def ext_cicloviario_por_bairro_ano(bairro_geom, ano):
    return round(sist_cicloviario.loc[sist_cicloviario.Ano_implantação == ano]['geometry'].apply(lambda x: intersection_lenght(x,bairro_geom)).sum(),2)

lista_anos = list(sist_cicloviario.dropna(subset = ['Ano_implantação'])['Ano_implantação'].unique())

lista = []
for index, row in gBairros_menor.iterrows():
    dados_bairro = {'bairro':row['bairro'],'faixa_idh': row['TIPO_IDH']}
    for ano in lista_anos:
        dados_bairro[ano] = ext_cicloviario_por_bairro_ano(row['the_geom'],ano)
        lista.append(dados_bairro)

df_bairros_malha_cicloviaria_por_ano = pd.DataFrame(lista)

In [74]:
malha_cicloviaria_idh_ano = df_bairros_malha_cicloviaria_por_ano.groupby('faixa_idh').sum().reset_index()
malha_cicloviaria_idh_ano.rename(columns={'atéjulho 2003':'2003','Não informado':'2022'}, inplace=True)

malha_cicloviaria_idh_ano.set_index('faixa_idh',inplace=True)
malha_cicloviaria_idh_ano = malha_cicloviaria_idh_ano.T
malha_cicloviaria_idh_ano.index = malha_cicloviaria_idh_ano.index.astype(int)

# Organizando a ordem das colunas: 
malha_cicloviaria_idh_ano = malha_cicloviaria_idh_ano.filter(["Muito Baixo", "Baixo","Médio","Alto","Muito Alto"])

# Calcula o valor cumulativo para o restante do dataset:  
# Se quiser remover o valor "Não informado" setado como -1, inserir: .iloc[1:,:]
malha_cicloviaria_idh_ano = malha_cicloviaria_idh_ano.sort_index().cumsum()

# Extensão absoluta por faixa de IDH:
malha_cicloviaria_idh_ano

faixa_idh,Muito Baixo,Baixo,Médio,Alto,Muito Alto
2003,316.54,99.62,0.00,73.44,1.87
2004,379.44,99.62,0.00,73.44,1.87
2007,498.95,99.62,0.00,73.44,1.87
2009,549.78,99.62,0.00,73.44,1.87
2010,566.78,99.62,0.00,73.44,1.87
2011,612.17,118.49,10.88,73.44,1.87
2012,884.85,147.73,10.88,73.44,1.87
2013,921.23,147.73,10.88,104.38,55.08
2014,1098.71,152.32,87.38,111.35,246.84
2015,1617.04,254.49,149.09,117.47,300.39


In [75]:
# Ext viária total por faixa de IDH em dicionário: 
totais = gBairros_menor.groupby('TIPO_IDH').sum()['ext_eixos_viarios'].to_dict()

In [76]:
totais

{'Alto': 98.83,
 'Baixo': 447.58,
 'Muito Alto': 171.47,
 'Muito Baixo': 3778.29,
 'Médio': 193.81}

In [77]:
# Para cada ano, calculamos a proporção da extensão atual do sistema cicloviário pela extensão total do eixo viário.

for c in malha_cicloviaria_idh_ano.columns:
    print(f'{c} - {totais[c]}')
    malha_cicloviaria_idh_ano[f'{c}(pct_total)'] = round(malha_cicloviaria_idh_ano[c]/totais[c]*100,2)

Muito Baixo - 3778.29
Baixo - 447.58
Médio - 193.81
Alto - 98.83
Muito Alto - 171.47


In [78]:
malha_cicloviaria_idh_ano.to_csv(arquivos_gerados+'malha_cicloviaria_idh_ano.csv')
malha_cicloviaria_idh_ano

faixa_idh,Muito Baixo,Baixo,Médio,Alto,Muito Alto,Muito Baixo(pct_total),Baixo(pct_total),Médio(pct_total),Alto(pct_total),Muito Alto(pct_total)
2003,316.54,99.62,0.00,73.44,1.87,8.38,22.26,0.00,74.31,1.09
2004,379.44,99.62,0.00,73.44,1.87,10.04,22.26,0.00,74.31,1.09
2007,498.95,99.62,0.00,73.44,1.87,13.21,22.26,0.00,74.31,1.09
2009,549.78,99.62,0.00,73.44,1.87,14.55,22.26,0.00,74.31,1.09
2010,566.78,99.62,0.00,73.44,1.87,15.00,22.26,0.00,74.31,1.09
2011,612.17,118.49,10.88,73.44,1.87,16.20,26.47,5.61,74.31,1.09
2012,884.85,147.73,10.88,73.44,1.87,23.42,33.01,5.61,74.31,1.09
2013,921.23,147.73,10.88,104.38,55.08,24.38,33.01,5.61,105.62,32.12
2014,1098.71,152.32,87.38,111.35,246.84,29.08,34.03,45.09,112.67,143.96
2015,1617.04,254.49,149.09,117.47,300.39,42.80,56.86,76.93,118.86,175.19


#### 3.6 Eixo Viário por faixa de IDH <a name='idh'></a>

In [79]:
eixos_viarios['Pavimentac'].unique()

array(['Sem', 'Asfalto', 'Paralelep.', 'Pedra rust', 'Rua Projetada',
       'Concreto', 'Premoldado'], dtype=object)

In [80]:
lista_eixos_por_pav = list(eixos_viarios['Pavimentac'].unique())
lista_eixos_por_pav

['Sem',
 'Asfalto',
 'Paralelep.',
 'Pedra rust',
 'Rua Projetada',
 'Concreto',
 'Premoldado']

In [81]:
# Essa função calcula a extensão do eixo viário de cada bairro por tipologia (pavimentado, não pavimentado, etc)

def ext_viario_por_bairro(bairro_geom, pav):
    return round(eixos_viarios.loc[eixos_viarios.Pavimentac == pav]['geometry'].apply(lambda x: intersection_lenght(x,bairro_geom)).sum(),2)

# Para cada bairro, verificamos a extensão de cada eixo viário considerando cada tipo de pavimentação

lista = []
for index, row in gBairros_menor.iterrows(): # Laço que percorre os bairros 
    dados_bairro = {'bairro':row['bairro'],'faixa_idh': row['TIPO_IDH']}
    for pav in lista_eixos_por_pav: # Laço que percorre a lista de tipos de pavimentação
        dados_bairro[pav] = ext_viario_por_bairro(row['the_geom'],pav)
    lista.append(dados_bairro)

df_bairros_por_pavimentacao = pd.DataFrame(lista)

In [82]:
df_bairros_por_pavimentacao.head() 

,bairro,faixa_idh,Sem,Asfalto,Paralelep.,Pedra rust,Rua Projetada,Concreto,Premoldado
0,Parque Araxá,Baixo,0.12,9.69,0.00,1.60,0.0,0.00,0.0
1,José Bonifácio,Médio,0.06,18.22,0.00,1.13,0.0,0.11,0.0
2,Parquelândia,Médio,0.06,27.22,0.00,0.53,0.0,0.00,0.0
3,Moura Brasil,Muito Baixo,0.09,6.03,0.38,2.48,0.0,0.00,0.0
4,Varjota,Alto,0.46,11.97,0.00,0.87,0.0,0.12,0.0


In [83]:
#Agrupa os bairros e suas respectivas pavimentações por faixa de IDH
pavimentacao_por_idh = df_bairros_por_pavimentacao.groupby('faixa_idh').sum().reset_index()
pavimentacao_por_idh['total_idh'] = pavimentacao_por_idh.sum(axis=1)

for pav in lista_eixos_por_pav:
    pavimentacao_por_idh[f'{pav}(%)'] = round(pavimentacao_por_idh[pav]/pavimentacao_por_idh['total_idh']*100,1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
pavimentacao_por_idh

,faixa_idh,Sem,Asfalto,Paralelep.,Pedra rust,Rua Projetada,Concreto,Premoldado,total_idh,Sem(%),Asfalto(%),Paralelep.(%),Pedra rust(%),Rua Projetada(%),Concreto(%),Premoldado(%)
0,Alto,3.36,83.92,2.05,9.10,0.10,0.12,0.18,98.83,3.4,84.9,2.1,9.2,0.1,0.1,0.2
1,Baixo,27.25,347.38,5.35,64.71,0.00,1.30,1.58,447.57,6.1,77.6,1.2,14.5,0.0,0.3,0.4
2,Muito Alto,2.13,164.80,1.84,2.65,0.03,0.00,0.00,171.45,1.2,96.1,1.1,1.5,0.0,0.0,0.0
3,Muito Baixo,657.91,2209.02,49.58,852.82,7.57,0.31,1.03,3778.24,17.4,58.5,1.3,22.6,0.2,0.0,0.0
4,Médio,1.62,178.27,0.30,12.37,1.15,0.11,0.00,193.82,0.8,92.0,0.2,6.4,0.6,0.1,0.0


In [85]:
pavimentacao_por_idh.sum()

faixa_idh           AltoBaixoMuito AltoMuito BaixoMédio
Sem                                              692.27
Asfalto                                         2983.39
Paralelep.                                        59.12
Pedra rust                                       941.65
Rua Projetada                                      8.85
Concreto                                           1.84
Premoldado                                         2.79
total_idh                                       4689.91
Sem(%)                                             28.9
Asfalto(%)                                        409.1
Paralelep.(%)                                       5.9
Pedra rust(%)                                      54.2
Rua Projetada(%)                                    0.9
Concreto(%)                                         0.5
Premoldado(%)                                       0.6
dtype: object

In [86]:
pavimentacao_por_idh.to_csv(arquivos_gerados+'pavimentacao_por_idh.csv', index=False)